# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [54]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
#from api_keys import geoapify_key
from config import geoapify_key

In [55]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fort mcmurray,56.7268,-111.3810,23.77,40,100,3.09,CA,1684705377
1,1,olinda,-8.0089,-34.8553,25.85,83,75,2.57,BR,1684705378
2,2,college,64.8569,-147.8028,20.05,39,40,2.57,US,1684705188
3,3,ancud,-41.8697,-73.8203,7.16,90,100,1.50,CL,1684705379
4,4,bredasdorp,-34.5322,20.0403,11.10,65,100,1.75,ZA,1684705379


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [56]:
%%capture --no-display

humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800, 
    frame_height = 600,
    size = "Humidity",
    scale = 1.3,
    color= "City")
        
#hover_cols = ["City", "Country", "Humidity"]
# Display the map plot
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_condition = (city_data_df['Max Temp']>21) & (city_data_df['Max Temp'] < 27) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)

# Drop any rows with null values
ideal_cities_df = city_data_df[ideal_weather_condition].dropna()

# Display sample data
print(ideal_cities_df)

In [ ]:
# response = requests.get(url).json()
# print(json.dumps(response, indent=4, sort_keys=True))

### Step 3: Create a new DataFrame called `hotel_df`.

In [58]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
0,fort mcmurray,CA,56.7268,-111.3810,40,
1,olinda,BR,-8.0089,-34.8553,83,
2,college,US,64.8569,-147.8028,39,
3,ancud,CL,-41.8697,-73.8203,90,
4,bredasdorp,ZA,-34.5322,20.0403,65,
...,...,...,...,...,...,...
548,opuwo,NaN,-18.0607,13.8400,39,
549,walvis bay,NaN,-22.9575,14.5053,93,
550,marsa alam,EG,25.0757,34.8918,37,
551,patacamaya,BO,-17.2333,-67.9167,18,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000

#set up parameters
params = {
    "categories" : "accommodation.hotel",
    "limit" : 20,
    "apiKey": geoapify_key
}

print("Starting hotel search")

# lat_range = (-90, 90)
# lng_range = (-180, 180)

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
#hotel_df
print(name_address)

Starting hotel search
fort mcmurray - nearest hotel: Nomad Hotel and Suites
olinda - nearest hotel: Fidji Motel
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
ancud - nearest hotel: Cabañas
bredasdorp - nearest hotel: Victoria Hotel
port-aux-francais - nearest hotel: Keravel
bang saphan - nearest hotel: Hotel
nikolayevsk-on-amure - nearest hotel: Север
avarua - nearest hotel: Paradise Inn
gadzhiyevo - nearest hotel: Оазис
yellowknife - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
adamstown - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
puerto leguizamo - nearest hotel: Hotel Girasoles
talnakh - nearest hotel: Талнах
montagu - nearest hotel: No hotel found
along - nearest hotel: Hotel Toshi Palace
port mathurin - nearest hotel: Escale Vacances
cidreira - nearest hotel: Hotel Castelo
nainpur - nearest hotel: No hotel found
pacific grove - nearest hotel: Pacific Grove Inn

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [65]:
%%capture --no-display

hotel_df_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800, 
    frame_height = 600,
    size = "Humidity",
    scale = 1.3,
    color= "City",
    hover_cols = ["Country", "hotel name"]    
)
# Display the map plot
hotel_df_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)